In [1]:
import pandas as pd

NROWS = 1000
n_input = 50

title_vec = pd.read_csv('hn_title_vec.csv', header=None, nrows=NROWS)
title_vec.rename(columns={0: 'id'}, inplace=True)

In [2]:
origin = pd.read_csv('/Users/Shared/data/HN_posts_year_to_Sep_26_2016.csv')
origin = origin[['id', 'num_points']]

In [3]:
sentence_length = pd.read_csv('sentence_length.csv', header=None, )
sentence_length.rename(columns={0: 'id', 1:'length'}, inplace=True)

In [4]:
data = pd.merge(title_vec, origin, how='inner', on=['id'])
data = pd.merge(data, sentence_length, how='inner', on=['id'])

In [5]:
GOOD_THRESHOLD = 3

data['good'] = data['num_points'].apply(lambda x: 1 if x >= GOOD_THRESHOLD else 0)
#data['good'].describe()

In [6]:
data = data.drop(['id', 'num_points', 'good'], 1)

In [7]:
for i in range(NROWS):
    for j in range(data.iloc[i]['length'].astype(int) * n_input + 1, data.shape[1]):
        data.set_value(i, j, 0)

In [8]:
train = data[list(range(1,data.shape[1]))]
seq_len = data['length']
seq_max_len = train.shape[1] // n_input
#train = data.sample(frac=0.8)
#test = data.drop(train.index)

In [9]:
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell

n_hidden = 100 # hidden layer num of features
n_classes = 50 # linear sequence or not

x = tf.placeholder("float", [None, seq_max_len * n_input])
seqlen = tf.placeholder(tf.int32, [None])

# Define weights
weights = {
    'encode': tf.Variable(tf.random_normal([n_hidden, n_classes])),
    'decode': tf.Variable(tf.random_normal([n_hidden, n_input]))
}
biases = {
    'encode': tf.Variable(tf.random_normal([n_classes])),
    'decode': tf.Variable(tf.random_normal([n_input]))
}

In [10]:
x1 = tf.reshape(x, [-1, seq_max_len, n_input])
x1 = tf.nn.l2_normalize(x1, 2)


# Permuting batch_size and n_steps, (n_step, batch_size, n_input)
#x1 = tf.transpose(x1, [1, 0, 2])
# Reshaping to (n_steps*batch_size, n_input)
#x2 = tf.reshape(x1, [-1, n_input])
# Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
#x3 = tf.split(0, seq_max_len, x2)

In [11]:
print(seqlen)
with tf.variable_scope('encode'):
    encoder_cell = tf.nn.rnn_cell.LSTMCell(n_hidden)
    encoded, encode_states = tf.nn.dynamic_rnn(encoder_cell, x1, dtype=tf.float32, sequence_length=seqlen)
    

Tensor("Placeholder_1:0", shape=(?,), dtype=int32)


In [12]:
#ones = tf.fill(tf.shape(seqlen), 1)
#e1 = tf.gather_nd(encoded, tf.subtract(seqlen, ones))
#batch_encoded = tf.matmul(e1, weights['encode']) + biases['encode']

In [ ]:
with tf.variable_scope('decode'):
    decoder_cell = tf.nn.rnn_cell.LSTMCell(n_hidden)
    decoder_input = tf.tile(encode_states.h, [1, seq_max_len])
    decoder_input = tf.reshape(decoder_input, [-1, seq_max_len, n_classes])
    decoded, decode_states = tf.nn.dynamic_rnn(decoder_cell, decoder_input, dtype=tf.float32, sequence_length=seqlen)
    
    d1 = tf.gather_nd(decoded, tf.subtract(seqlen, ones))
    batch_decoded = tf.matmul(d1, weights['decode']) + biases['decode']

In [ ]:
# decoded: n_step list of (batch_size, n_input)
# y: (n_step*batch_size, n_input)
#y = tf.concat(0, decoded)

# y2: (n_step, batch_size, n_input)
#y2 = tf.reshape(y, [seq_max_len, -1, n_input])

# y3: (batch_size, n_step, n_input)
#y3 = tf.transpose(y2, [1, 0, 2])

y3 = tf.nn.l2_normalize(decoded, 2)
y3 = tf.reshape(y3, [-1, seq_max_len* n_input])

In [ ]:
# Parameters
learning_rate = 0.1

# Define loss and optimizer
cost = tf.reduce_mean(tf.square(tf.sub(y3, x)))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [ ]:
# Initializing the variables
init = tf.initialize_all_variables()

epouch = 100
display_step = 10;
batch_size = 1;

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    for i in range(epouch):
        for j in range(NROWS // batch_size):
        #for j in range(len(train)):
            #feed = train.iloc[j].as_matrix().reshape(1, seq_max_len, n_input)
            #feed = train.as_matrix().reshape(NROWS, seq_max_len, n_input)
            feed = train.iloc[range(j*batch_size, (j+1)*batch_size)].as_matrix()
            slen = seq_len[j*batch_size: (j+1)*batch_size]
            sess.run(optimizer, feed_dict={x: feed, seqlen: slen})
            if j % display_step == 0:
                loss = sess.run(cost, feed_dict={x: feed, seqlen: slen})
                print("Epouch" + str(i) + ", step" + str(j) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss))
                enc = sess.run(encode_states, feed_dict={x: feed, seqlen: slen})
                out = sess.run(y3, feed_dict={x: feed, seqlen: slen})
                print(feed)
                print(enc.h)
                print(out)
    print("Optimization Finished!")